In [13]:
!pip install autogluon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error

 ### 데이터 불러오기

In [4]:
fall = pd.read_csv("/content/drive/MyDrive/날씨 빅데이터 공모전/data/fall_train.csv")

fall_test = pd.read_csv("/content/drive/MyDrive/날씨 빅데이터 공모전/data/fall_test.csv")

### mmddhh 칼럼 월 일 시간으로 나누기

In [5]:
fall["hour"] = fall["mmddhh"] % 100
fall["day"] = (np.trunc((fall["mmddhh"] % 10000) / 100))
fall["month"] = np.trunc(fall["mmddhh"] / 10000)

fall_test["hour"] = fall_test["mmddhh"] % 100
fall_test["day"] = (np.trunc((fall_test["mmddhh"] % 10000) / 100))
fall_test["month"] = np.trunc(fall_test["mmddhh"] / 10000)

### 지면온도 예측에 대한 RN(누적강수량)의 일부 값을 변경

In [6]:
fall = fall.replace({"1시간 누적 강수량(mm)" : [1.8, 3.3, 5.3, 19.5, 30.3, 623.5]}, -99.9)

fall_test = fall_test.replace({"1시간 누적 강수량(mm)" : [1.8, 3.3, 5.3, 19.5, 30.3, 623.5]}, -99.9)

### 1시간 누적 일사량과 1시간 누적 일조량 칼럼 수정

In [7]:
fall_8 = [0, 1, 2, 3, 4, 5, 21, 22, 23] #6시도 결측치인 경우가 있어 일부분을 수정
fall_9 = [0, 1, 2, 3, 4, 5, 6, 20, 21, 22, 23]
fall_10 = [0, 1, 2, 3, 4, 5, 6, 19, 20, 21, 22, 23]

for i in fall_8 :
    fall.loc[(fall["1시간 누적 일사량(MJ)"] == -99.9) & (fall["hour"] == i) & (fall["월"] == 8), "1시간 누적 일사량(MJ)"] = 0
    fall.loc[(fall["1시간 누적 일조량(초)"] == -99.9) & (fall["hour"] == i) & (fall["월"] == 8), "1시간 누적 일조량(초)"] = 0

for i in fall_9 :
    fall.loc[(fall["1시간 누적 일사량(MJ)"] == -99.9) & (fall["hour"] == i) & (fall["월"] == 9), "1시간 누적 일사량(MJ)"] = 0
    fall.loc[(fall["1시간 누적 일조량(초)"] == -99.9) & (fall["hour"] == i) & (fall["월"] == 9), "1시간 누적 일조량(초)"] = 0

for i in fall_10 :
    fall.loc[(fall["1시간 누적 일사량(MJ)"] == -99.9) & (fall["hour"] == i) & (fall["월"] == 10), "1시간 누적 일사량(MJ)"] = 0
    fall.loc[(fall["1시간 누적 일조량(초)"] == -99.9) & (fall["hour"] == i) & (fall["월"] == 10), "1시간 누적 일조량(초)"] = 0


# 8월 6시도 결측치인 경우가 있어 일부분을 수정
fall.loc[(fall["1시간 누적 일사량(MJ)"] == -99.9) & (fall["hour"] == 6) & (fall["월"] == 8), "1시간 누적 일사량(MJ)"] = 0
fall.loc[(fall["1시간 누적 일조량(초)"] == -99.9) & (fall["hour"] == 6) & (fall["월"] == 8), "1시간 누적 일조량(초)"] = 0

# 8월 20시의 경우도 마찬가지
fall.loc[(fall["1시간 누적 일사량(MJ)"] == -99.9) & (fall["hour"] == 20) & (fall["월"] == 8), "1시간 누적 일사량(MJ)"] = 0
fall.loc[(fall["1시간 누적 일조량(초)"] == -99.9) & (fall["hour"] == 20) & (fall["월"] == 8), "1시간 누적 일조량(초)"] = 0


for i in fall_8 :
    fall_test.loc[(fall_test["1시간 누적 일사량(MJ)"] == -99.9) & (fall_test["hour"] == i) & (fall_test["월"] == 8), "1시간 누적 일사량(MJ)"] = 0
    fall_test.loc[(fall_test["1시간 누적 일조량(초)"] == -99.9) & (fall_test["hour"] == i) & (fall_test["월"] == 8), "1시간 누적 일조량(초)"] = 0

for i in fall_9 :
    fall_test.loc[(fall_test["1시간 누적 일사량(MJ)"] == -99.9) & (fall_test["hour"] == i) & (fall_test["월"] == 9), "1시간 누적 일사량(MJ)"] = 0
    fall_test.loc[(fall_test["1시간 누적 일조량(초)"] == -99.9) & (fall_test["hour"] == i) & (fall_test["월"] == 9), "1시간 누적 일조량(초)"] = 0

for i in fall_10 :
    fall_test.loc[(fall_test["1시간 누적 일사량(MJ)"] == -99.9) & (fall_test["hour"] == i) & (fall_test["월"] == 10), "1시간 누적 일사량(MJ)"] = 0
    fall_test.loc[(fall_test["1시간 누적 일조량(초)"] == -99.9) & (fall_test["hour"] == i) & (fall_test["월"] == 10), "1시간 누적 일조량(초)"] = 0


fall.loc[(fall["1시간 누적 일사량(MJ)"] == -99.9) & (fall["1시간 누적 일조량(초)"] == 0), "1시간 누적 일사량(MJ)"] = 0
fall.loc[(fall["1시간 누적 일사량(MJ)"] == 0) & (fall["1시간 누적 일조량(초)"] == -99.9), "1시간 누적 일조량(초)"] = 0

fall_test.loc[(fall_test["1시간 누적 일사량(MJ)"] == -99.9) & (fall_test["1시간 누적 일조량(초)"] == 0), "1시간 누적 일사량(MJ)"] = 0
fall_test.loc[(fall_test["1시간 누적 일사량(MJ)"] == 0) & (fall_test["1시간 누적 일조량(초)"] == -99.9), "1시간 누적 일조량(초)"] = 0

### 결측치 처리

In [8]:
fall = fall.replace([-99, -99.9, -999], np.nan)

fall_test = fall_test.replace([-99, -99.9, -999], np.nan)

### 사용하지 않는 칼럼 제거

In [9]:
fall = fall.drop(["mmddhh", "적설 깊이(cm)", "월"], axis = 1)

fall_test = fall_test.drop(["mmddhh", "적설 깊이(cm)", "월"], axis = 1)

### 1시간 누적 강수유무와 1시간 누적 강수량 칼럼 수정

In [10]:
fall.loc[(fall["1시간 누적 강수유무(분)"]== 0) & (fall["1시간 누적 강수량(mm)"].isnull()), "1시간 누적 강수량(mm)"] = 0
fall.loc[fall["1시간 누적 강수량(mm)"].isnull(), "1시간 누적 강수량(mm)"] = 0
fall = fall.drop("1시간 누적 강수유무(분)", axis = 1)

fall_test.loc[(fall_test["1시간 누적 강수유무(분)"]== 0) & (fall_test["1시간 누적 강수량(mm)"].isnull()), "1시간 누적 강수량(mm)"] = 0
fall_test.loc[fall_test["1시간 누적 강수량(mm)"].isnull(), "1시간 누적 강수량(mm)"] = 0
fall_test = fall_test.drop("1시간 누적 강수유무(분)", axis = 1)

### 인덱스 초기화

In [11]:
fall = fall.reset_index(drop=True)

### 라벨 인코딩 수행

In [12]:
cha = ["지점", "연도", "현천"]

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in cha:
    fall[i] = le.fit_transform(fall[i])

    for label in np.unique(fall_test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)

    fall_test[i] = le.transform(fall_test[i])

<ipython-input-12-84aa01043848>:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if label not in le.classes_:



## 파생변수 추가

### 1. 기온 변화량

In [14]:
# 파생변수 추가
fall = fall.reset_index(drop = True)
fall_test = fall_test.reset_index(drop=True)

fall["기온변화량"] = -(fall["1시간 평균 기온(C)"] - fall["1시간 평균 기온(C)"][1:].reset_index(drop = True))
fall["기온변화량"] = fall["기온변화량"].shift(1)

fall_test["기온변화량"] = -(fall_test["1시간 평균 기온(C)"] - fall_test["1시간 평균 기온(C)"][1:].reset_index(drop = True))
fall_test["기온변화량"] = fall_test["기온변화량"].shift(1)


### 2. 상대습도 변화량

In [15]:
fall["상대습도변화량"] = -(fall["1시간 평균 상대습도(%)"] - fall["1시간 평균 상대습도(%)"][1:].reset_index(drop = True))
fall["상대습도변화량"] = fall["상대습도변화량"].shift(1)

fall_test["상대습도변화량"] = -(fall_test["1시간 평균 상대습도(%)"] - fall_test["1시간 평균 상대습도(%)"][1:].reset_index(drop = True))
fall_test["상대습도변화량"] = fall_test["상대습도변화량"].shift(1)


### 3. 평균 기온 * 평균 상대습도

In [16]:
fall["평균기온*평균상대습도"] = fall["1시간 평균 기온(C)"] * fall["1시간 평균 상대습도(%)"]

fall_test["평균기온*평균상대습도"] = fall_test["1시간 평균 기온(C)"] * fall_test["1시간 평균 상대습도(%)"]


### 4. 1시간 누적 일사량 * 1시간 누적 일조량

In [17]:
fall["1시간 누적 일사량 * 1시간 누적 일조량"] = fall["1시간 누적 일사량(MJ)"] * fall["1시간 누적 일조량(초)"]

fall_test["1시간 누적 일사량 * 1시간 누적 일조량"] = fall_test["1시간 누적 일사량(MJ)"] * fall_test["1시간 누적 일조량(초)"]

### 2개 이상 연속되는 결측치 찾기 및 제거

### 결측치 확인

In [18]:
fall.isnull().sum()

지점                           0
연도                           0
1시간 평균 기온(C)               189
1시간 평균 이슬점온도(C)            192
1시간 평균 상대습도(%)             167
1시간 평균풍속(m/s)              328
1시간 누적 강수량(mm)               0
현천                           0
1시간 평균 지면온도(C)             183
1시간 누적 일사량(MJ)             616
1시간 누적 일조량(초)              238
hour                         0
day                          0
month                        0
기온변화량                      273
상대습도변화량                    240
평균기온*평균상대습도                190
1시간 누적 일사량 * 1시간 누적 일조량    626
dtype: int64

### - 2개 이상 연속되는 결측치 찾기 및 제거

In [19]:
lists = ["1시간 평균 기온(C)", "1시간 평균 이슬점온도(C)", "1시간 평균 상대습도(%)", "1시간 평균풍속(m/s)" , "1시간 평균 지면온도(C)", "1시간 누적 일사량(MJ)", "1시간 누적 일조량(초)"
,"기온변화량" ,"상대습도변화량", "평균기온*평균상대습도", "1시간 누적 일사량 * 1시간 누적 일조량"]
for i in lists:
    ans = fall[fall[i].isnull()].index
    cons = []
    for j in range(len(ans)-1) :
        if ans[j+1] - ans[j] == 1 : # i+1번째 숫자와 i번째 숫자의 차이가 1이라면(연속이라면)
            cons.extend([ans[j],ans[j+1]]) #i+1번째 숫자와 i번째 숫자를 결과 출력할 리스트에 추가
    delete = list(set(cons)) #위 알고리즘으로 하면 만일 숫자가 23,24,25라면 23, 24, 24, 25로 저장되기 때문에 중복되는 숫자를 제거

    fall = fall.drop(fall.index[delete])
    #인덱스를 초기화
    fall = fall.reset_index(drop=True)

In [20]:
fall

,지점,연도,1시간 평균 기온(C),1시간 평균 이슬점온도(C),1시간 평균 상대습도(%),1시간 평균풍속(m/s),1시간 누적 강수량(mm),현천,1시간 평균 지면온도(C),1시간 누적 일사량(MJ),1시간 누적 일조량(초),hour,day,month,기온변화량,상대습도변화량,평균기온*평균상대습도,1시간 누적 일사량 * 1시간 누적 일조량
0,0,0,24.2,24.1,99.6,0.8,0.0,0,24.0,0.0,0.0,0,1.0,8.0,NaN,NaN,2410.32,0.0
1,0,0,24.0,24.0,99.6,0.6,0.0,3,24.0,0.0,0.0,1,1.0,8.0,-0.2,-0.0,2390.40,0.0
2,0,0,23.8,23.8,99.9,0.7,0.0,3,23.8,0.0,0.0,2,1.0,8.0,-0.2,0.3,2377.62,0.0
3,0,0,23.7,23.7,99.8,0.7,0.0,0,23.8,0.0,0.0,3,1.0,8.0,-0.1,-0.1,2365.26,0.0
4,0,0,23.8,23.7,99.6,0.6,0.0,3,23.9,0.0,0.0,4,1.0,8.0,0.1,-0.2,2370.48,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109481,9,4,17.9,10.9,63.7,1.4,0.0,0,15.9,0.0,0.0,19,31.0,10.0,-1.1,7.8,1140.23,0.0
109482,9,4,17.1,11.0,67.1,2.3,0.0,0,15.0,0.0,0.0,20,31.0,10.0,-0.8,3.4,1147.41,0.0
109483,9,4,16.8,11.0,68.9,2.7,0.0,0,14.5,0.0,0.0,21,31.0,10.0,-0.3,1.8,1157.52,0.0
109484,9,4,16.0,10.4,69.5,2.4,0.0,0,14.0,0.0,0.0,22,31.0,10.0,-0.8,0.6,1112.00,0.0


### 결측치 채우기 : 선형 보간

In [21]:
fall = fall.interpolate(method = 'linear')

fall_test = fall_test.interpolate(method = 'linear')

### autogluon 적용

In [22]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [23]:
# 0~2년도의 데이터를 train, 3년도의 데이터를 validation, 4년도의 데이터를 test로 하여 autogluon 적용

#1번 방법
auto_fall_train = fall.loc[(fall["연도"] != 3) & (fall["연도"] != 4)]
auto_fall_val = fall.loc[fall["연도"] == 3]
auto_fall_test = fall.loc[fall["연도"] == 4]

In [24]:
#1번 방법
predictor = TabularPredictor(label="1시간 평균 지면온도(C)", eval_metric = "mae").fit(auto_fall_train, tuning_data = auto_fall_val)

No path specified. Models will be saved in: "AutogluonModels/ag-20230705_044639/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230705_044639/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Apr 29 09:15:28 UTC 2023
Disk Space Avail:   83.44 GB / 115.66 GB (72.1%)
Train Data Rows:    65662
Train Data Columns: 17
Tuning Data Rows:    22010
Tuning Data Columns: 17
Label Column: 1시간 평균 지면온도(C)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (65.1, -2.0, 23.0801, 8.96067)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preproces

In [25]:
predictor.leaderboard(auto_fall_test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,-1.230213,-1.410313,0.160866,0.021821,7.838792,0.160866,0.021821,7.838792,1,True,6
1,XGBoost,-1.259394,-1.379160,0.147892,0.094692,3.699281,0.147892,0.094692,3.699281,1,True,9
2,LightGBMLarge,-1.262371,-1.415993,0.358046,0.335753,5.532017,0.358046,0.335753,5.532017,1,True,11
3,LightGBMXT,-1.267731,-1.458920,0.252021,0.309269,5.892363,0.252021,0.309269,5.892363,1,True,3
4,ExtraTreesMSE,-1.283614,-1.439775,3.182015,1.896737,57.804182,3.182015,1.896737,57.804182,1,True,7
5,WeightedEnsemble_L2,-1.287526,-1.338383,5.490316,2.495484,306.828741,0.008782,0.003365,1.072965,2,True,12
6,LightGBM,-1.290539,-1.398397,0.129661,0.371323,5.033894,0.129661,0.371323,5.033894,1,True,4
7,RandomForestMSE,-1.308481,-1.424298,4.344996,1.666479,186.383225,4.344996,1.666479,186.383225,1,True,5
8,NeuralNetTorch,-1.624206,-1.904826,0.184758,0.115554,50.363667,0.184758,0.115554,50.363667,1,True,10
9,KNeighborsDist,-1.872932,-2.032150,6.518404,18.111703,0.063706,6.518404,18.111703,0.063706,1,True,2


### fall_test에 적용

In [26]:
fall_pred = predictor.predict(fall_test)

In [27]:
fall_pred

0       25.196815
1       25.079035
2       25.085606
3       25.209745
4       25.151848
          ...    
6619    15.056761
6620    14.319115
6621    13.932985
6622    13.744524
6623    13.620214
Name: 1시간 평균 지면온도(C), Length: 6624, dtype: float32

In [28]:

import pandas as pd

# 파일명
file_name = '/content/drive/MyDrive/날씨 빅데이터 공모전/data/subminssionUser.xlsx'

writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
submission_fall = pd.read_excel(file_name, sheet_name= "AUTUMN")

# Daraframe형식으로 엑셀 파일 읽기
submission_fall = pd.read_excel(file_name, sheet_name= "AUTUMN")

#제출 파일에 빠진 날짜 찾아서 테스트에서도 빼버리기
submission_fall["TS"] = fall_pred

submission_fall.to_excel(
    writer,
    sheet_name='AUTUMN',
    index=False,
    # header = None
    )
writer.save()
writer.close()

<ipython-input-28-c1d66ada0e12>:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
